In [1]:
#Import pandas
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cd ../..

/Users/arthur/Documents/Flatiron/phase_1/Project/main_microsoft_movies/microsoft_movies


In [3]:
#read in the relevant data files
basics_df = pd.read_csv("data/imdb.title.basics.csv")
ratings_df = pd.read_csv("data/imdb.title.ratings.csv")
principals_df = pd.read_csv("data/imdb.title.principals.csv")
imdb_name_df = pd.read_csv("data/imdb.name.basics.csv")

In [4]:
#remove all rows without a genre
basics_df.dropna(subset=['genres'], inplace=True)

In [5]:
#coverted the genres column, which has genres separated by columns into
#list of strings
basics_df['genres'] = basics_df['genres'].str.strip().str.split(',')

In [6]:
#create a new df with the list of genres separated out into individual rows
e_basics_df = basics_df.explode('genres')
e_basics_df

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Action
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Crime
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,Drama
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,Biography
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,Drama
...,...,...,...,...,...,...
146138,tt9916428,The Secret of China,The Secret of China,2019,NaN,War
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy


In [7]:
f_basics_df = e_basics_df[(e_basics_df['genres'] == 'Drama') |
                                 (e_basics_df['genres'] == 'Action') |
                                 (e_basics_df['genres'] == 'Adventure') |
                                 (e_basics_df['genres'] == 'Comedy') ]

In [8]:
#merge the e_basics_df with the ratings_df
combined_df = pd.merge(f_basics_df, ratings_df, how='inner', on='tconst')

In [9]:
combined_df = pd.merge(combined_df, principals_df, on='tconst', how='inner')
combined_df = pd.merge(combined_df, imdb_name_df, on='nconst', how='inner')

In [10]:
combined_df['start_year'].sort_values().unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [11]:
combined_df = combined_df[(combined_df['start_year'] > 2014)]

In [12]:
combined_df['category'].unique()

array(['writer', 'actress', 'actor', 'director', 'composer',
       'cinematographer', 'editor', 'producer', 'production_designer',
       'self', 'archive_footage', 'archive_sound'], dtype=object)

In [13]:
combined_df = combined_df[(combined_df['category'] == 'actor') |
                          (combined_df['category'] == 'actress') |
                          (combined_df['category'] == 'director') |
                          (combined_df['category'] == 'writer')]

In [14]:
combined_df = combined_df.sort_values(['averagerating', 'numvotes'], ascending=(False, False))

In [15]:
#print out various stats for number of votes per movie to
#determine threshold
print(combined_df['numvotes'].max())
print(combined_df['numvotes'].min())
print(combined_df['numvotes'].median())
print(combined_df['numvotes'].mean())

820847
5
102.0
6270.205703368242


In [16]:
#chose the mean as the threshold and filtered out any number of votes
#below the mean
combined_df = combined_df[(combined_df['numvotes'] > combined_df['numvotes'].mean())]

In [17]:
comb_drama_df = combined_df[(combined_df['genres'] == 'Drama')]
comb_action_df = combined_df[(combined_df['genres'] == 'Action')]
comb_adventure_df = combined_df[(combined_df['genres'] == 'Adventure')]
comb_comedy_df = combined_df[(combined_df['genres'] == 'Comedy')]

In [18]:
comb_action_df[(comb_action_df['category'] == 'director')]

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,ordering,nconst,category,job,characters,primary_name,birth_year,death_year,primary_profession,known_for_titles
123658,tt5813916,The Mountain II,Dag II,2016,135.0,Action,9.3,100568,5,nm3809021,director,NaN,NaN,Alper Caglar,NaN,NaN,"producer,writer,editor","tt2471640,tt3345952,tt5813916,tt8197998"
209239,tt4154796,Avengers: Endgame,Avengers: Endgame,2019,181.0,Action,8.8,441135,5,nm0751577,director,NaN,NaN,Anthony Russo,1970.0,NaN,"producer,director,writer","tt1843866,tt4154756,tt3498820,tt4154796"
209247,tt4154796,Avengers: Endgame,Avengers: Endgame,2019,181.0,Action,8.8,441135,6,nm0751648,director,NaN,NaN,Joe Russo,1971.0,NaN,"producer,director,writer","tt4154756,tt4154796,tt1843866,tt3498820"
400232,tt7060344,Ratsasan,Ratsasan,2018,170.0,Action,8.8,10518,5,nm6442107,director,NaN,NaN,Ram Kumar,NaN,NaN,"writer,director,actor","tt9042396,tt3681442,tt7060344"
176215,tt6148156,Vikram Vedha,Vikram Vedha,2017,147.0,Action,8.7,18704,5,nm2566836,director,NaN,NaN,Gayatri,NaN,NaN,"director,writer,costume_designer","tt6148156,tt0977625,tt1296162,tt1735932"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205163,tt5456546,Judwaa 2,Judwaa 2,2017,145.0,Action,3.9,7136,5,nm0223522,director,NaN,NaN,David Dhawan,NaN,NaN,"director,editor,writer","tt0115009,tt0807758,tt0418362,tt0172188"
200379,tt6513656,Escape Plan 2: Hades,Escape Plan 2: Hades,2018,96.0,Action,3.8,20280,5,nm1921345,director,NaN,NaN,Steven C. Miller,1981.0,NaN,"director,editor,writer","tt2347497,tt6513656,tt3963816,tt1816597"
354014,tt2976182,Roy,Roy,2015,146.0,Action,3.3,6681,5,nm6379626,director,NaN,NaN,Vikramjit Singh,NaN,NaN,"writer,cinematographer,director","tt4985254,tt2976182,tt0083452"
46081,tt7825208,Marighella,Marighella,2019,155.0,Action,2.8,41882,5,nm0609944,director,NaN,NaN,Wagner Moura,1976.0,NaN,"actor,producer,soundtrack","tt0861739,tt1555149,tt7825208,tt1535108"
